<h1>Data Processing</h1>

### Einleitung und Motivation

##### Einleitung

Ziel dieses Notebooks ist, die gescrapeten Daten so zu verarbeiten, dass Sie später thematisch durchsucht werden können. Ein Embeddingvektor liefert genau diese Funktion. Auf eine Anfrage hin wird der Abstand zwischen dem Embedding der Frage und den Embeddings aller anderen Dokumenten berechnet. Dann werden die Dokumente mit den kleinsten Abständen ausgewählt und dem LLM als Kontext mitgegeben. Mithilfe des Wissens dieser Dokumente soll dass LLM dann in der Lage sein die Frage korrekt zu beantworten.
Für die Embeddings benutzen wir [Google Bert](https://blog.google/products/search/search-language-understanding-bert/)

Beispiel:

User: Welche Dozenten unterrichten das Fach Grundlagen der Informatik?

System wählt besten 5 Dokumente aus 

> <Dokument 1>: ... betreute Prof. Dr. Löhr eine Batchelorarbeit in Grundlagen der Informatik... <br>
> <Dokument 2>: Prof. Dr. Weber tel.: 013882664 email: weber@th.de Raum: HQ: 403, Fächer: Grundlagen der Informatik ... <br>
> <Dokument 3> ... <br>
> <Dokument 4> ... <br>
> <Dokument 5> ... <br>
    

Aus der Nutzeranfrage und den Dokumenten wird eine neue Query erstellt, die dem LLM dann final bereitgetellt wird. Diese sieht in etwa so aus:

    
> <Dokument 1>: ... betreute Prof. Dr. Löhr eine Batchelorarbeit in Grundlagen der Informatik... <br>
> <Dokument 2>: Prof. Dr. Weber tel.: 013882664 email: weber@th.de Raum: HQ: 403, Fächer: Grundlagen der Informatik ... <br>
> <Dokument 3> ... <br>
> <Dokument 4> ... <br>
> <Dokument 5> ... <br>
> Bitte beantworte folgende Frage unter der Berücksichtigung obiger Dokumente:
> Welche Dozenten unterrichten das Fach Grundlagen der Informatik?


Das LLM wird daraufhin hoffentlich korrekt eine Antwort liefern die ähnlich ist zu:

> A: An der TH Nürnberg Georg Simon Ohm unterichten die Professoren Prof. Dr. Löhr und Prof. Dr. Weber das Fach Grundlagen der Informatik.


##### imports

In [ ]:
import ast
from db_init import db_get_df, db_save_df
import json

# import gensim
import matplotlib.pyplot as plt
import numpy as np
import random
import pandas as pd
from embedding_algorithms.question_embedding import question_embeddings
from scipy.spatial.distance import cosine
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import torch
from transformers import BertModel, BertTokenizer
from tqdm import tqdm
from wordcloud import WordCloud
from sentence_transformers import SentenceTransformer

### Laden der Daten

##### Laden aus der sqlite db

Zunächst laden wir die Daten aus der Datenbank. Dabei besitzt jedes Dokument als Metadaten den Titel der Webseite, den filenamen und den Text. Diese speichern wir uns in einen Pandas Dataframe

In [ ]:
df = db_get_df("html_attrs", ["filename", "title", "text"])

print(df.dtypes)
print(df["text"][3])

##### Filtern der Daten

Wir sortieren zunächst alle Dokumete aus, die keinen Text beinhalten

In [ ]:
print(f"Ungefiltert sind es {len(df)} Dokumente")
df = df[df["text"].apply(len) != 0]
df.reset_index(drop=True, inplace=True)
print(f"gefiltert sind es {len(df)} Dokumente")

In [ ]:
df["filename"]

##### nach Sprachen filtern

Jetzt filtern wir die Seiten noch in Englisch und deutsche Seiten

In [ ]:
df_en = df[df['filename'].str.startswith('data/htmlfiles/file_en')]
len(df_en)

In [ ]:
df_de = df[df['filename'].str.startswith('data/htmlfiles/file_en') == False]
len(df_de)

##### Beispiel Keywort suche

Zur überprüfung der Texte können wir nun einmal eine Keywordsuche starten. Dieser Ansatz wird außerdem tiefer im Notebook [spacy_keywordextraction](./spacy_keywordextraction.ipynb) verfolgt.

In [ ]:
word = "Gallwitz"

[text for text in df["text"] if word in text][:5]

### Embeddings berechnen

##### Einleitung zu BERT Tokens

Jetzt werden wir für jedes Dokument ein eigenes Word embeddings erstellen. Dazu müssen wir zunächst das BERT Model laden. <br>
Das BERT Model ist ein von Google trainierter Encoder, welcher ursprünglich dafür Entwickelt wurde, dass er maskierte Wörter erraten kann (Masked Language Modelling) oder Vorhersagen kann, ob ein Satz auf einen anderen Satz folgt.(Next Sentence Prediction)
- es ist trainiert auf 10.000+ Büchern
- es gibt Modelle "base" und "large"
uncased heißt ohne klein - Großschreibung

Wir brauchen zur Vorbereitung die zusätzlichen Token
- [SEP] um das Ende eines Satzes zu markieren
- [CLS] am Anfang des Texten
- [PAD] zum auffüllen der Token 
Außerdem
TokenIDs
MaskIDs - zum filtern der [PAD]
Segment IDs um verschiedene Sätze zu unterscheiden
Posititional Embeddings


##### Laden des BERT Modells und des BERT tokenizers

In [ ]:
#TODO try better model
# BertModel.from_pretrained('bert-base-german-cased',output_hidden_states = True) 
model = BertModel.from_pretrained('bert-base-uncased',output_hidden_states = True) 
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

##### Testen des Tokenizers

In [ ]:
testSentence = "In der Bibliothek gibt es 40 Bücher zum Thema Animes"
tokens_question = tokenizer.tokenize(testSentence)
print(*tokens_question)

##### Tokens im dataframe speichern

Nun erstellen wir eine neue Spalte ["tokens"], in der wir für jedes Dokument die Tokens abspeichern.

In [ ]:
df["tokens"] = [tokenizer.tokenize(text) for text in tqdm(df["text"])]

Der dataframe hat nun eine Spalte mehr und wir können uns ein Beispiel der Tokens ansehen.

In [ ]:
print(df.dtypes)
print(df["tokens"][2])

##### Tokens in IDs umwandeln

Diese Tokens müssen nun in IDs umgewandelt werden, damit sie das BERT Model für die Erstellung eines Embedding Vectors benutzen kann. Dafür benutzen wir eine Funktion des Tokenizers ```tokenizer.convert_tokens_to_ids()```

In [ ]:
df["token_ids"] = [tokenizer.convert_tokens_to_ids(tokens) for tokens in tqdm(df["tokens"])]

In [ ]:
special_symbols = ["[CLS]", "[SEP]", "[PAD]"]
print(tokenizer.convert_tokens_to_ids(special_symbols))


##### Tokens splitten

Wir haben jetzt also die Tokens IDs für unsere 2400 verschiedenen Dokumente gebildet. Der nächste Schritt wäre nun, diese Token IDs dem BERT Model zu übergeben, sodass es uns ein Embedding daraus errechnet. Leider kann das BERT Model nur 512 Tokens (~1300 Zeichen) als Input nehmen. Die meißten der gescrapeten Webseiten sind aber wesentlich länger. <br> 
Der Naheliegendste Ansatz ist dabei, die Tokens einfach in 510 token große Chunks aufzusplitten (Wir brauchen noch 2 Tokens extra für jeden Chunk) und für jeden Chunk ein extra Embedding zu erstellen. <br> 
Dabei gibt es entweder die Möglichkeit die Chunks überlappend, oder einfach hard cut zu gestalten. <br>

Wir werden hier zunächst den hard cut Ansatz verfolgen. 

In [ ]:

def split_text_and_tokens(row):
    text = row['text']
    tokens_ids = row['token_ids']
    filename = row['filename']

    if len(tokens_ids) > 510:
        for i in range(0, len(tokens_ids), 510):
            chunk_tokens = tokens_ids[i:i + 510]
            # adding the [CLS] and the [SEP] token
            chunk_tokens = [101] + chunk_tokens + [102]
            chunk_text = tokenizer.decode(chunk_tokens)

            new_row = {'filename': filename, 'chunk_id': i/510, 'chunk_text': chunk_text, 'chunk_tokens_json': json.dumps(chunk_tokens)}
            new_rows.append(new_row)
    else:
        # adding the [CLS] and the [SEP] token
        tokens_ids = [101] + tokens_ids + [102]
        tokens_ids = tokens_ids + [0] * (512 -len(tokens_ids))
        text = "[CLS]" + text + "[SEP]"
        new_row = {'filename': filename, 'chunk_id': 0, 'chunk_text': text, 'chunk_tokens_json': json.dumps(tokens_ids) }
        new_rows.append(new_row)

new_rows = []
df.apply(split_text_and_tokens, axis=1)
chunk_df = pd.DataFrame(new_rows)
chunk_df.reset_index(drop=True, inplace=True)

Wir können uns an dieser Stelle die gesplitteten chunks anschauen

In [ ]:
print(chunk_df.sample(2).to_markdown())

Ab jetzt werden wir mit dem neuen chunk_df weiterarbeiten. Nun erstellen wir für jeden Chunk ein eigenes Embedding, welches dann die Semantik dieses chunks enthalten soll. Dafür müssen wir uns nun das BERT Model etwas genauer anschauen.

In [ ]:
def proccessSentence(tokens, model, tokenizer):
    if len(tokens) == 0:
        return torch.zeros(768)

    tokens = ["CLS"] + tokens + ["SEP"]

    attention_mask = [1 if token != "[PAD]" else 0  for token in tokens]
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    token_ids_tensor = torch.tensor([token_ids], dtype=torch.int64)
    attetion_mask_tensor = torch.tensor([attention_mask], dtype=torch.int64)

    with torch.no_grad():
        outputs = model(token_ids_tensor, attetion_mask_tensor)
        hidden_states = outputs[2]

    # stack the layer list 
    token_embeddings = torch.stack(hidden_states, dim=0)
    # remove the batches dim
    token_embeddings = torch.squeeze(token_embeddings, dim=1)
    # Swap dimensions 0 and 1.
    token_embeddings = token_embeddings.permute(1,0,2)
    # average all token embeds
    layer_vecs = torch.mean(token_embeddings, dim=0)



    # Calculate the average of layer 3 to 13
    embed = torch.mean(layer_vecs[2:], dim=0)


    return embed

df["chunk_embeddings"] = [proccessSentence(tokens).tolist() for tokens in tqdm(df["tokens"])]


Diser Code kann sehr lange brauchen um die Embeddings zu berechnen. Der Code An dieser Stelle sollte man dann die Embeddings am besten abspeichern. 

db_save_df(df, "chunk_embeddings")

##### T-SNE

In [ ]:
df = db_get_df("chunk_embeddings", ["*"])

In [ ]:

def embed_t_sne(embed_id):
    df_embed=[json.loads(embedding) for embedding in tqdm(df[f"chunk_embeddings_{embed_id}"])]
    word_embeddings = np.array(df_embed)
    tsne = TSNE(n_components=2, perplexity=30, n_iter=300)
    X_embedded = tsne.fit_transform(word_embeddings)


    plt.figure(figsize=(10, 6))
    plt.scatter(X_embedded[:, 0], X_embedded[:, 1], s=5)
    plt.title(f"t-SNE Visualization of Word Embeddings {embed_id}")
    plt.xlabel("t-SNE Dimension 1")
    plt.ylabel("t-SNE Dimension 2")
    plt.show() 

In [ ]:
for i in range(5):
    embed_t_sne(i +1)

In [ ]:
# Funktion zur Extraktion von Word Embeddings für die Frage
def get_word_embedding(question, model_name='bert-base-uncased'):
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertModel.from_pretrained(model_name)

    tokens = tokenizer(question, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**tokens)
        question_embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    return question_embedding

# Laden der Word Embeddings
df = db_get_df("chunk_embeddings", ["chunk_embeddings_2"])
df = [json.loads(embedding) for embedding in tqdm(df["chunk_embeddings_2"])]
word_embeddings = np.array(df)

# Extrahieren der Embeddings für die Frage
question_text = "n?"
question_embedding = get_word_embedding(question_text)

# Berechnen der Kosinus-Ähnlichkeit zwischen der Frage und den anderen Word Embeddings
similarities = cosine_similarity(word_embeddings, [question_embedding])

# 'similarities' ist jetzt ein Array mit den Kosinus-Ähnlichkeiten zwischen der Frage und den anderen Word Embeddings.

# Kombinieren Sie die t-SNE-Komponenten mit den Kosinus-Ähnlichkeiten
combined_features = np.column_stack((X_embedded, similarities))

# Visualisierung
plt.figure(figsize=(10, 6))
plt.scatter(combined_features[:, 0], combined_features[:, 1], s=5)
plt.scatter(question_embedding[0], question_embedding[1], color='red', s=50, label='Ihre Frage')
plt.title("t-SNE Visualization of Word Embeddings with Cosine Similarity")
plt.xlabel("t-SNE Dimension 1")
plt.ylabel("t-SNE Dimension 2")
plt.legend()
plt.show()


In [ ]:
# Annahme: X_embedded ist Ihre t-SNE-Visualisierung
# Annahme: word_embeddings ist Ihre Matrix der Word Embeddings
# Annahme: question_text ist Ihre Frage
# Annahme: n_clusters ist die Anzahl der gewünschten Cluster
question_text = "Welche Kompetenzen hat Pr. Gallwitz?" #wann ist der Bewerbungszeitraum  Für das Wintersemester

# Schritt 1: Clustering durchführen
kmeans = KMeans(n_clusters=5, random_state=0)
cluster_labels = kmeans.fit_predict(word_embeddings)

# Schritt 3: Berechnen der Ähnlichkeit zur Frage
question_embedding = get_word_embedding(question_text)  # Verwenden Sie Ihre get_word_embedding Funktion
similarities = cosine_similarity(word_embeddings, [question_embedding])

# Schritt 4: Visualisierung aktualisieren
plt.figure(figsize=(10, 6))
for i in range(5):
    plt.scatter(X_embedded[cluster_labels == i, 0], X_embedded[cluster_labels == i, 1], s=5, label=f'Cluster {i}')

# Farben entsprechend des Clusters für die Frage aktualisieren
question_cluster = np.argmax(similarities)
plt.scatter(X_embedded[question_cluster, 0], X_embedded[question_cluster, 1], s=50, color='red', label='Ihre Frage')

plt.title("t-SNE Visualization of Word Embeddings with Clusters")
plt.xlabel("t-SNE Dimension 1")
plt.ylabel("t-SNE Dimension 2")
plt.legend()
plt.show()


In [ ]:

kmeans = KMeans(n_clusters=5)  # Specify the number of clusters you want
cluster_labels = kmeans.fit_predict(word_embeddings)


In [ ]:
plt.figure(figsize=(10, 6))
for i in range(len(np.unique(cluster_labels))):
    plt.scatter(X_embedded[cluster_labels == i, 0], X_embedded[cluster_labels == i, 1], s=5, label=f'Cluster {i}')

plt.title("t-SNE Visualization of Word Embeddings with Clusters")
plt.xlabel("t-SNE Dimension 1")
plt.ylabel("t-SNE Dimension 2")
plt.legend()
plt.show()

In [ ]:
# Filter the indices of data points in Cluster 0
cluster0_indices = np.where(cluster_labels == 1)

# Get the corresponding rows from the DataFrame 'df'
cluster0_data_rows = df.iloc[cluster0_indices]

# Print the 'text' column for the data points in Cluster 0
for text in cluster0_data_rows['text']:
    print(text)


In [ ]:


# Annahme: Ihr DataFrame 'df' enthält eine Spalte 'text' mit den Textdaten.

# Anzahl der Cluster (angenommen, es sind 5 Cluster)
num_clusters = 5

for cluster_id in range(num_clusters):
    # Filtern Sie die Zeilen für den aktuellen Cluster
    cluster_data_rows = df

    # Laden des spaCy-Modells für die Textverarbeitung
    nlp = spacy.load("de_core_news_sm")

    # Benutzerdefinierte Stoppwortliste
    stopwords = {'www', 'th-nuernberg', 'nürnberg', 'nuernberg', 'th', 'technische', 'hochschule', 'ohm', 'de', 'punkt', 'simon'}

    # Tokenisieren und Lemmatisieren der Texte, Entfernen der Stoppwörter und Konvertieren in Strings
    processed_texts = []
    for text in cluster_data_rows['text']:
        doc = nlp(text)
        processed_tokens = []
        for token in doc:
            if token.text.lower() not in stopwords and token.pos_ in {'NOUN', 'PROPN'}:
                processed_tokens.append(token.text)
        processed_texts.append(' '.join(processed_tokens))

    # Erstellen eines Wörterbuchs und einer Textkorpus für das LDA-Modell
    text_tokens = [text.split() for text in processed_texts]
    dictionary = gensim.corpora.Dictionary(text_tokens)
    corpus = [dictionary.doc2bow(tokens) for tokens in text_tokens]

    # Anwendung des LDA-Modells
    lda_model = gensim.models.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15)

    # Anzeigen der Hauptthemen für den aktuellen Cluster
    print(f"Cluster {cluster_id} Topics:")
    for topic_id, topic in lda_model.print_topics():
        print(f"Topic {topic_id}: {topic}")
    print("\n")


In [ ]:


all_text = ' '.join(processed_texts)  # 'processed_texts' ist die Liste der bereinigten Texte

# Erstellen der Word Cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(all_text)

# Anzeigen der Word Cloud
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title("Word Cloud")
plt.show()

In [ ]:


# Erstellen einer Liste von Stoppwörtern, einschließlich der URL und der benutzerdefinierten Wörter
stopwords = set(['www', 'th-nuernberg', 'nürnberg', 'nuernberg', 'th', 'technische', 'hochschule', 'ohm', 'de', 'punkt', 'simon','https','http','nuremberg','telefon','email','fax','Prof Dr','studium'])
stopwords = set(word.lower() for word in stopwords)  # In Kleinbuchstaben umwandeln

# Anzahl der Cluster (angenommen, es sind 5 Cluster)
num_clusters = 5

for cluster_id in range(num_clusters):
    if cluster_id == 0:
        continue
    # Filter the indices of data points in the current cluster
    cluster_indices = np.where(cluster_labels == cluster_id)

    # Get the corresponding rows from the DataFrame 'df'
    cluster_data_rows = df.iloc[cluster_indices]

    # Extract and preprocess text data
    texts = cluster_data_rows['text']
    nlp = spacy.load("de_core_news_sm")
    processed_texts = [' '.join([token.text for token in nlp(text) if not token.is_stop and token.text.lower() not in stopwords]) for text in texts]

    # Create a Word Cloud for the current cluster
    all_text = ' '.join(processed_texts)
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(all_text)

    # Display the Word Cloud for the current cluster
    plt.figure(figsize=(10, 6))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(f"Word Cloud for Cluster {cluster_id}")
    plt.show()


In [ ]:
### TF-IDF Algorithmus

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

question="Welche Kompetenzen hat Prf. Gallwitz?"

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['text'])
question_tfidf = tfidf_vectorizer.transform([question])
similarities = cosine_similarity(question_tfidf, tfidf_matrix)

similarity_df = pd.DataFrame({
    'Similarity': similarities[0],
    'Text': df['text']
})
sorted_similarity_df = similarity_df.sort_values(by='Similarity', ascending=False)

# Print the top N most relevant documents (e.g., top 5)
top_n = 5
relevant_documents = sorted_similarity_df.head(top_n)

# Print the relevant documents and their similarities to the question
for index, row in relevant_documents.iterrows():
    print(f"Similarity: {row['Similarity']}")
    print(row['Text'])
    print('-' * 50)

In [ ]:
#dokument--> bert anwenden für jeden dokument
# question-bert anwenden
question="was macht Gallwitz?"
document=df["text"][1]

tokens_question = tokenizer.tokenize(question)
tokens_document = tokenizer.tokenize(document)
attetion_mask_question = [1] * len(tokens_question)
attention_mask_dokument = [1] * len(tokens_document)

token_idss = tokenizer.convert_tokens_to_ids(tokens_question)
tokenDocument_idss = tokenizer.convert_tokens_to_ids(tokens_document)


tokens_tensor = torch.tensor([token_idss])
segments_tensors = torch.tensor([attetion_mask_question])

tokensDocument_tensor = torch.tensor([tokenDocument_idss])
segmentsDocument_tensors = torch.tensor([attention_mask_dokument])

with torch.no_grad():
    outputs = model(tokens_tensor, segments_tensors)
    hidden_states = outputs[2]

with torch.no_grad():
    outputs = model(tokensDocument_tensor, segmentsDocument_tensors)
    hiddenDocuments_states = outputs[2]

# print(token_idss)
# print(tokenDocument_idss)

print ("Number of layers:", len(hidden_states), "  (initial embeddings + 12 BERT layers)")
layer_i = 0

print ("Number of batches:", len(hidden_states[layer_i]))
batch_i = 0

print ("Number of tokens:", len(hidden_states[layer_i][batch_i]))
token_i = 0

print ("Number of hidden units:", len(hidden_states[layer_i][batch_i][token_i]))


In [ ]:
token_embeddings = torch.stack(hidden_states, dim=0)
token_embeddings = torch.squeeze(token_embeddings, dim=1)
token_embeddings = token_embeddings.permute(1,0,2)

token_vecs_sum = []

# For each token in the sentence...
for token in token_embeddings:
    
    # Sum the vectors from the last four layers.
    sum_vec = torch.sum(token[-4:], dim=0)
    
    # Use `sum_vec` to represent `token`.
    token_vecs_sum.append(sum_vec)

print ('Shape is: %d x %d' % (len(token_vecs_sum), len(token_vecs_sum[0])))

In [ ]:
token_vecs = hidden_states[-2][0]

# Calculate the average of all 22 token vectors.
sentence_embedding = torch.mean(token_vecs, dim=0)
print ("Our final sentence embedding vector of shape:", sentence_embedding)


tokenDocuments_vecs = hiddenDocuments_states[-2][0]

# Calculate the average of all 22 token vectors.
sentenceDocument_embedding = torch.mean(tokenDocuments_vecs, dim=0)
print ("Our final sentence embedding vector of shape:", sentence_embedding)


In [ ]:
# Calculate the cosine similarity between the word bank 
# in "bank robber" vs "river bank" (different meanings).
diff_bank = 1 - cosine(sentence_embedding, sentenceDocument_embedding)

print('Vector similarity for *different* meanings:  %.2f' % diff_bank)

In [ ]:
word = "Fachhochschulgesetz"

df.loc[df["text"].str.contains(word)]["text"]
# [text for text in df["text"] if word in text][:5]

In [ ]:
df = db_get_df("chunk_embeddings")
questions = ["Was besagt das Fachhochschulgesetz?", 
             "Wo befindet sich die Mensa", 
             "Welche Professoren gibt es an der Technischen Hochschule Nürnberg?",
             "Gib mir alle Infos zum Studienstart",
             "Was gibt es Neues im Bezug auf Künstliche Intelligenz an der Hochschule?",
             "Themen für eine Batchelorarbeit",
             "Where is the Language office?",
             "Where can i find the Mensa",
             "Give me information on beginning of Semester",
             "How many professors are there at the TH?"]
question = questions[random.randint(0,9)]
question_embedding = question_embeddings(question)

df["distance"] = [1 - cosine(json.loads(embedding), question_embedding) for embedding in df["chunk_embeddings_2"]]
most_similar_documents = df.nsmallest(5, "distance")
# print(f"question embedding: {question_embedding[:10]}")
print(question)
print(most_similar_documents["chunk_text"].to_markdown())


df["distance"].plot(kind='hist', bins=200)
plt.show()

In [ ]:
df = db_get_df("word_embeddings", ["filename", "title", "text", "tokens"])
df["token_ids"] = [tokenizer.convert_tokens_to_ids(json.loads(tokens)) for tokens in df["tokens"]]

In [ ]:
# splice dokuments in 512 token chunks

# Initialize an empty list to store rows for the new DataFrame
new_rows = []

# Function to split text and tokens into chunks of 512 tokens
def split_text_and_tokens(row):
    text = row['text']
    tokens_ids = row['token_ids']
    filename = row['filename']

    if len(tokens_ids) > 512:
        # Split into multiple chunks
        for i in range(0, len(tokens_ids), 512):
            chunk_tokens = tokens_ids[i:i + 512]
            chunk_text = tokenizer.decode(chunk_tokens)

            # Create a new row with a reference to the original row
            new_row = {'filename': filename, 'chunk_id': i/512, 'chunk_text': chunk_text, 'chunk_tokens_json': json.dumps(chunk_tokens)}
            new_rows.append(new_row)
    else:
        # If the row has 512 tokens or fewer, keep it as is
        new_row = {'filename': filename, 'chunk_id': 0, 'chunk_text': text, 'chunk_tokens_json': json.dumps(tokens_ids) }
        new_rows.append(new_row)

# Apply the function to each row in the original DataFrame
df.apply(split_text_and_tokens, axis=1)

# Create a new DataFrame from the list of new rows
new_df = pd.DataFrame(new_rows)

# Reset the index of the new DataFrame if needed
new_df.reset_index(drop=True, inplace=True)

# Print the new DataFrame
print(new_df.to_markdown())

# tokenizer.convert_tokens_to_string


Aus den 2433 Dokumenten die wir eigentlich gescraped haben, sind nun 6945 chunks entstanden es hat sich fast verdreifacht. Wenn man die 787 Seiten ohne Inhalt abzieht, hat sich die Anzahl von 1646 auf 6158 fast vervierfacht.

In [ ]:
new_df["chunk_tokens_json"][2]

In [ ]:
db_save_df(new_df, "chunk_word_embeddings")

In [ ]:
import sqlite3
from dotenv import load_dotenv
import os

load_dotenv()
database_path = os.getenv("DATABASE_PATH")

def merge_db_tables():
    # Connect to your SQLite database
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()
    # Create the new table using the structure of the first table (chunk_word_embeddings_0)
    cursor.execute('''CREATE TABLE chunk_word_embeddings_all AS SELECT * FROM chunk_word_embeddings_0 WHERE 0''')
    # Insert data from the other tables into the new table
    for i in range(1, 8):
        cursor.execute(f'INSERT INTO chunk_word_embeddings_all SELECT * FROM chunk_word_embeddings_{i}')
    # Commit the changes and close the connection
    conn.commit()
    conn.close()

merge_db_tables()

### Calculate embeddings with SentenceTransformer from huggingface

##### paraphrase-MiniLM-L6-v2 with spacy

In [ ]:
import spacy
from sentence_transformers import SentenceTransformer
import pandas as pd
df = db_get_df("html_attrs_de", ["filename", "title", "text"])
# Load spaCy for sentence segmentation
nlp = spacy.load("de_core_news_sm")
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
#TODO speichere embeddings in databank
sentences = []
embeddings = []

for text in df['text']:
    # Split the text into sentences using spaCy
    doc = nlp(text)
    sentence_list = [sent.text for sent in doc.sents]
    sentences.append(sentence_list)
    
    # Encode each sentence using Sentence Transformers
    sentence_embeddings = [model.encode(sentence) for sentence in sentence_list]
    embeddings.append(sentence_embeddings)
    print(embeddings)

# Now, 'sentences' is a list where each element is a list of sentences, and 'embeddings' is a list of corresponding sentence embeddings.
# filenames = df.head(3)
# print(filenames.to_markdown())



##### embeddings with sentence-transformers/all-MiniLM-L6-v2 model

In [ ]:
from sentence_transformers import SentenceTransformer
import pandas as pd

# Load the pre-trained MiniLM model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Create a list to store the embeddings
embeddings = []

# Assuming 'text' column holds your HTML content
for text in df['text']:
    # Encode the text and append the resulting embedding to the list
    text_embedding = model.encode(text)
    embeddings.append(text_embedding)

# Create a new DataFrame to store the embeddings
embeddings_df1 = pd.DataFrame(embeddings)

# Now 'embeddings_df' contains the embeddings for each text in your DataFrame
db_save_df(embeddings_df1, "embeddings1")


##### embeddings with 'sentence-transformers/msmarco-distilbert-base-tas-b' model

In [ ]:
from sentence_transformers import SentenceTransformer
import pandas as pd
from tqdm import tqdm

# Load the pre-trained Multi-QA MiniLM model
model = SentenceTransformer('sentence-transformers/msmarco-distilbert-base-tas-b')

# Create a list to store the embeddings
embeddings = []

# Assuming 'text' column holds your HTML content
for text in tqdm(df['text'], desc="Encoding Texts"):
    # Encode the text and append the resulting embedding to the list
    text_embedding = model.encode(text)
    embeddings.append(text_embedding)

# Create a new DataFrame to store the embeddings
embeddings_df3 = pd.DataFrame(embeddings)
db_save_df(embeddings_df3, "embeddings3")

# Now 'embeddings_df' contains the embeddings for each text in your DataFrame using the multi-qa model


##### embeddings with 'sentence-transformers/multi-qa-MiniLM-L6-cos-v1' model

In [ ]:
from sentence_transformers import SentenceTransformer
import pandas as pd
from tqdm import tqdm

# Load the pre-trained Multi-QA MiniLM model
model = SentenceTransformer('sentence-transformers/multi-qa-MiniLM-L6-cos-v1')

# Create a list to store the embeddings
embeddings = []

# Assuming 'text' column holds your HTML content
for text in tqdm(df['text'], desc="Encoding Texts"):
    # Encode the text and append the resulting embedding to the list
    text_embedding = model.encode(text)
    embeddings.append(text_embedding)

# Create a new DataFrame to store the embeddings
embeddings_df4 = pd.DataFrame(embeddings)
db_save_df(embeddings_df4, "embeddings4")

# Now 'embeddings_df' contains the embeddings for each text in your DataFrame using the multi-qa model


##### Calculate embeddings with 'paraphrase-MiniLM-L6-v2' model 

Wir laden das Modell "paraphrase-MiniLM-L6-v2" von HuggingFace herunter. Daraus werden Embeddings für die jeweiligen TH-Webseiten berechnet. Das Modell erstellt Embeddings der Größe 384 Floats.

In [ ]:

df = db_get_df("html_attrs", ["filename", "title", "text"])
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
text_embeddings = []

for index, row in tqdm(df.iterrows(), total=len(df), desc="Calculating embeddings"):
    text_embedding = model.encode(row['text'])
    text_embeddings.append({'filename': row['filename'],
                            'title': row['title'],
                            'text': row['text'],
                            'text_embedding': text_embedding})

embeddings_df = pd.DataFrame(text_embeddings)



Sqlite kann keine Listen speichern, daher werde hier json Objekte erstellt. 

In [ ]:
embeddings_df["text_embedding_json"] = [json.dumps(text_embedding.tolist()) for text_embedding in embeddings_df["text_embedding"]]
db_save_df(embeddings_df[["filename","title","text","text_embedding_json" ]], "embeddings_paraphrase_MiniLM_L6_v2")

In [ ]:
def calculate_similarities(query,model,embeddings_df):
    query_embedding = model.encode(query, show_progress_bar=True)
    similarities = []

    for index, row in tqdm(embeddings_df.iterrows(), total=len(embeddings_df), desc="Calculating similarities"):
        
        similarity = cosine_similarity([query_embedding], [row["text_embedding"]])[0][0]

        similarities.append({'filename': row['filename'],
                            'title': row['title'],
                            'text': row['text'], 
                            'similarity': similarity})

    similarities_df = pd.DataFrame(similarities)
    return similarities_df




In [ ]:
def plot_similarities(similarities_df):
    similarities_df['similarity'].plot(kind='hist', bins=200)
    plt.xlabel('Similarity')
    plt.ylabel('Frequency')
    plt.title('Similarity Distribution')
    plt.show()

Wir schauen uns die similarities für unterschiedliche Fragen an.

In [ ]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
embeddings_df=db_get_df("embeddings_paraphrase_MiniLM_L6_v2")
# Convert back to lists.
embeddings_df["text_embedding"] = [json.loads(text_embedding_json) for text_embedding_json in embeddings_df["text_embedding_json"]]
queries = ["Was besagt das Fachhochschulgesetz?", 
             "Wo befindet sich die Mensa", 
             "Welche Professoren gibt es an der Technischen Hochschule Nürnberg?",
             "Gib mir alle Infos zum Studienstart",
             "Was gibt es Neues im Bezug auf Künstliche Intelligenz an der Hochschule?",
             "Themen für eine Batchelorarbeit",
             "Where is the Language office?",
             "Where can i find the Mensa",
             "Give me information on beginning of Semester",
             "How many professors are there at the TH?"]

most_similar_articles=""

for query in queries:
    similarities_df=calculate_similarities(query,model,embeddings_df)
    most_similar_articles = similarities_df.nlargest(5, 'similarity')['text']
    print("prompt: ", query)
    print("most_similar_articles: \n"+most_similar_articles.to_markdown())
    plot_similarities(similarities_df)

### Combine Intranet HTML and TH-Nuernberg.de

In [3]:
import pandas as pd

In [1]:
from db_init import db_get_df, db_save_df

df_th_nuernberg = db_get_df("html_attrs_2086")
df_intranet = db_get_df("intranet_html_attrs_1686")

In [2]:
print(df_th_nuernberg.dtypes)
print(df_intranet.dtypes)

link     object
html     object
text     object
title    object
dtype: object
link     object
html     object
text     object
title    object
dtype: object


In [4]:
df = pd.concat([df_th_nuernberg, df_intranet], ignore_index=True)

In [7]:
db_save_df(df, "html_attr_combined")